## To DO

- remove CL:0000533 from the hand-annotated data - it's not in the census
- finish internal cell accuracy calculations
- predict better(?) internal cells annotations

In [1]:
import anndata as ad
import torch
import torch.nn as nn
import torch.nn.functional as F


import pandas as pd
import numpy as np
import scipy 

import scanpy

import os


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
ad.__version__

'0.10.3'

In [3]:
scanpy.__version__

'1.9.3'

## Load the hand annotated data

In [3]:
adata_ann = ad.read_h5ad('multivelo.h5ad') # hand annotated 7th day uninfected HSPC - variable genes only

# hand annotated 7th day uninfected HSPC  - original cellranger output matrix containing all cells and genes
#adata = ad.read_h5ad('filtered_feature_bc_matrix.h5') 
adata_raw = scanpy.read_10x_h5('filtered_feature_bc_matrix.h5')#,key='matrix')

/home/fujoshua/cell_classification/env/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/fujoshua/cell_classification/env/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [25]:
adata_ann


AnnData object with n_obs × n_vars = 11605 × 899
    obs: 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'S_score', 'G2M_score', 'phase', 'fractions_u', 'leiden'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'mean', 'std', 'fit_alpha_c', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_sw1', 'fit_t_sw2', 'fit_t_sw3', 'fit_scale_cc', 'fit_rescale_c', 'fit_rescale_u', 'fit_alignment_scaling', 'fit_c0', 'fit_u0', 'fit_s0', 'fit_model', 'fit_direction', 'fit_loss', 'fit_likelihood', 'fit_anchor_min_idx', 'fit_anchor_max_idx', 'velo_s_genes', 'velo_u_genes', 'velo_chrom_genes'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap', 'velo_chrom_params', 'velo_s_params', 'velo_u_params'
    obsm: 'X_pca', 'X_

In [52]:
adata_raw

AnnData object with n_obs × n_vars = 14895 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [34]:
adata_raw.obs#.index.nunique()

""
AAACAGCCAAACCTTG-1
AAACAGCCACCCTCAC-1
AAACATGCAATCGCAC-1
AAACATGCACAGCCTG-1
AAACATGCAGAGGCTA-1
...
TTTGTTGGTGTTGTGA-1
TTTGTTGGTTACCTGT-1
TTTGTTGGTTCCGGGA-1
TTTGTTGGTTGCCTCA-1


In [43]:
adata_ann.obs

,n_counts,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,initial_size_spliced,initial_size_unspliced,initial_size,S_score,G2M_score,phase,fractions_u,leiden
AAACAGCCAAACCTTG-1,4234.334961,1497,3384.0,0.0,0.0,3384,1563,3384.0,-0.634671,-0.215746,G1,0.308031,Granulocyte
AAACAGCCACCCTCAC-1,4246.441406,1673,3251.0,0.0,0.0,3251,1904,3251.0,0.423275,-0.362225,S,0.301643,Erythrocyte
AAACATGCAATCGCAC-1,4228.449219,1652,3272.0,0.0,0.0,3272,1849,3272.0,-0.801623,-0.463818,G1,0.374815,Erythrocyte
AAACATGCACAGCCTG-1,4223.146973,3637,10265.0,0.0,0.0,10265,3769,10265.0,0.408826,0.540197,G2M,0.290607,Prog MK
AAACATGCAGAGGCTA-1,4230.780762,1572,3526.0,0.0,0.0,3526,1792,3526.0,0.400493,-0.121910,S,0.318393,MPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTTAAAC-1,4194.219238,1642,3244.0,0.0,0.0,3244,1723,3244.0,0.000000,-0.201697,S,0.340409,HSC
TTTGTTGGTGTTGTGA-1,4216.825195,2306,5480.0,0.0,0.0,5480,2348,5480.0,-0.291098,-0.117548,G1,0.345615,MEP
TTTGTTGGTTACCTGT-1,4240.065430,1664,3310.0,0.0,0.0,3310,1678,3310.0,0.000000,0.155516,G2M,0.304211,MEP
TTTGTTGGTTCCGGGA-1,4253.901855,2193,5188.0,0.0,0.0,5188,1870,5188.0,0.296031,0.044591,S,0.322785,Erythrocyte


In [15]:
adata_ann

AnnData object with n_obs × n_vars = 11605 × 899
    obs: 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'S_score', 'G2M_score', 'phase', 'fractions_u', 'leiden'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'mean', 'std', 'fit_alpha_c', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_sw1', 'fit_t_sw2', 'fit_t_sw3', 'fit_scale_cc', 'fit_rescale_c', 'fit_rescale_u', 'fit_alignment_scaling', 'fit_c0', 'fit_u0', 'fit_s0', 'fit_model', 'fit_direction', 'fit_loss', 'fit_likelihood', 'fit_anchor_min_idx', 'fit_anchor_max_idx', 'velo_s_genes', 'velo_u_genes', 'velo_chrom_genes'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap', 'velo_chrom_params', 'velo_s_params', 'velo_u_params'
    obsm: 'X_pca', 'X_

In [ ]:
# what exactly do I need in terms of meta data?
- GENE ENSEMBL IDs - merged_data.var['gene_ids']
- Cell Type - merged_data.obs['leiden']

## Merge the Raw Data File with the Annotated file

In [4]:
def merge_data_files(adata_raw,adata_ann):
    '''
    Function that 
    
    adata_raw : AnnData Object
        original cellranger output matrix containing all cells and genes
        
    adata_ann : AnnData Object
        hand annotated data, variable genes only
    
    '''
    
    # the index of the obs axis of the AnnData objects contains strings
    # that identify each individual cell. 
    # we want to create a new AnnData Object that includes:
    # 1) all gene expressions from adata_raw
    # 2) only the cells in adata_ann
    # 3) the var and obs axes from adata_ann
    
    # get list of cells from the annotated data
    cell_list = adata_ann.obs.index

    # selects the cells in the annotated file from the raw file
    merged_data = adata_raw[cell_list]

    # add the obs axis from the annotated data to the merged data 
    merged_data.obs = adata_ann.obs
    
    return merged_data


In [5]:
merged_data = merge_data_files(adata_raw,adata_ann)

/home/fujoshua/cell_classification/env/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/fujoshua/cell_classification/env/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
merged_data

AnnData object with n_obs × n_vars = 11605 × 36601
    obs: 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'S_score', 'G2M_score', 'phase', 'fractions_u', 'leiden'
    var: 'gene_ids', 'feature_types', 'genome'

## Map the annotated cell types to Cell Ontology IDs

What we need are indicies that we can then use to split the data later on. 

In [7]:
merged_data_obs_df = merged_data.obs#['leiden']#.unique().tolist()

In [8]:
merged_data_obs_df

,n_counts,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,initial_size_spliced,initial_size_unspliced,initial_size,S_score,G2M_score,phase,fractions_u,leiden
AAACAGCCAAACCTTG-1,4234.334961,1497,3384.0,0.0,0.0,3384,1563,3384.0,-0.634671,-0.215746,G1,0.308031,Granulocyte
AAACAGCCACCCTCAC-1,4246.441406,1673,3251.0,0.0,0.0,3251,1904,3251.0,0.423275,-0.362225,S,0.301643,Erythrocyte
AAACATGCAATCGCAC-1,4228.449219,1652,3272.0,0.0,0.0,3272,1849,3272.0,-0.801623,-0.463818,G1,0.374815,Erythrocyte
AAACATGCACAGCCTG-1,4223.146973,3637,10265.0,0.0,0.0,10265,3769,10265.0,0.408826,0.540197,G2M,0.290607,Prog MK
AAACATGCAGAGGCTA-1,4230.780762,1572,3526.0,0.0,0.0,3526,1792,3526.0,0.400493,-0.121910,S,0.318393,MPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTTAAAC-1,4194.219238,1642,3244.0,0.0,0.0,3244,1723,3244.0,0.000000,-0.201697,S,0.340409,HSC
TTTGTTGGTGTTGTGA-1,4216.825195,2306,5480.0,0.0,0.0,5480,2348,5480.0,-0.291098,-0.117548,G1,0.345615,MEP
TTTGTTGGTTACCTGT-1,4240.065430,1664,3310.0,0.0,0.0,3310,1678,3310.0,0.000000,0.155516,G2M,0.304211,MEP
TTTGTTGGTTCCGGGA-1,4253.901855,2193,5188.0,0.0,0.0,5188,1870,5188.0,0.296031,0.044591,S,0.322785,Erythrocyte


In [9]:
# double check at least PROG DC, this was the primary one that was
# unclear in identifying

# all of these values vall under Hematopoietic cell
# 553 and 1012 not in cell census

mapping_dict = {'Granulocyte' : 'CL:0000094',
                'Erythrocyte' : 'CL:0000232',
                'Prog MK' : 'CL:0000553',
                'MPP' : 'CL:0000837',
                'GMP' : 'CL:0000557',
                'HSC' : 'CL:0000037',
                'LMPP' : 'CL:0000936',
                'MEP' : 'CL:0000050',
                'Prog DC' : 'CL:0001012',
                'Platelet' : 'CL:0000233',
                'Prog B' : 'CL:0000826'}    
    

merged_data_obs_df['cell_type'] = merged_data_obs_df['leiden'].map(mapping_dict)

In [10]:
cell_type_list = merged_data_obs_df['cell_type'].to_list()

# loop through the list and save the index values

leaf_value_list = ['CL:0000936']
internal_value_list = ['CL:0000094','CL:0000232','CL:0000553','CL:0000837','CL:0000557','CL:0000037',
                      'CL:0000050','CL:0001012','CL:0000233','CL:0000826']

leaf_index_values = []
internal_index_values = []

for itr in range(len(cell_type_list)):
    if cell_type_list[itr] in leaf_value_list:
        leaf_index_values.append(itr)
    else:
        internal_index_values.append(itr)
        

In [29]:
leaf_targets = [cell_type_list[i] for i in leaf_index_values] 
internal_targets = [cell_type_list[i] for i in internal_index_values]

In [71]:
leaf_targets

['CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:0000936',
 'CL:00009

This data has 11605 cells with 899 genes measured for each. 

### Var Dataframe

In [27]:
adata.var

,Accession,Chromosome,End,Start,Strand,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,...,fit_s0,fit_model,fit_direction,fit_loss,fit_likelihood,fit_anchor_min_idx,fit_anchor_max_idx,velo_s_genes,velo_u_genes,velo_chrom_genes
Gene,,,,,,,,,,,,,,,,,,,,,
ABAT,ENSG00000183044,16,8784575,8674596,+,False,272,0.020566,98.057698,288.0,...,0.000000e+00,1.0,on,0.000651,0.053900,45.0,482.0,True,True,True
ABCA1,ENSG00000165029,9,104928155,104781006,-,False,245,0.018566,98.250500,260.0,...,5.823158e-05,1.0,complete,0.001142,0.039279,86.0,365.0,False,False,False
ABCC3,ENSG00000108846,17,50692253,50634777,+,False,108,0.010783,99.228792,151.0,...,8.646782e-04,1.0,on,0.000871,0.008601,20.0,482.0,False,False,False
ABCC4,ENSG00000125257,13,95301475,95019835,-,False,6723,1.259212,51.992288,17634.0,...,1.366475e-01,1.0,on,0.173222,0.507818,53.0,484.0,True,True,True
ABHD2,ENSG00000140526,15,89202355,89087459,+,False,6018,0.726078,57.026564,10168.0,...,2.874029e-05,1.0,complete,0.073722,0.097848,67.0,498.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF780B,ENSG00000128000,19,40056231,40028260,-,False,981,0.073693,92.994859,1032.0,...,1.833241e-05,1.0,on,0.002445,0.042543,348.0,487.0,False,False,False
ZNF783,ENSG00000204946,7,149297302,149262171,+,False,542,0.040988,96.129677,574.0,...,0.000000e+00,1.0,on,0.001448,0.065151,215.0,487.0,True,True,True
ZNF785,ENSG00000197162,16,30585769,30573740,-,False,81,0.005784,99.421594,81.0,...,2.132494e-05,1.0,complete,0.000157,0.016822,36.0,242.0,False,False,False


In [28]:
var_df = adata.var

In [25]:
var_df['Accession'].nunique()

899

So there are 899 genes in this dataset 

In [32]:
var_df.index.to_list()

['ABAT',
 'ABCA1',
 'ABCC3',
 'ABCC4',
 'ABHD2',
 'ABLIM1',
 'AC004069.1',
 'AC004540.2',
 'AC004825.3',
 'AC005498.3',
 'AC005856.1',
 'AC006116.8',
 'AC007262.2',
 'AC008875.2',
 'AC008875.3',
 'AC009061.2',
 'AC009126.1',
 'AC009318.1',
 'AC009495.3',
 'AC009509.1',
 'AC009879.4',
 'AC010737.1',
 'AC010745.5',
 'AC010896.1',
 'AC011246.1',
 'AC011472.2',
 'AC016735.1',
 'AC018695.9',
 'AC020656.1',
 'AC021092.1',
 'AC023510.1',
 'AC026369.3',
 'AC074032.1',
 'AC090579.1',
 'AC090630.1',
 'AC091563.1',
 'AC092919.1',
 'AC096667.1',
 'AC096711.2',
 'AC096751.2',
 'AC097376.3',
 'AC111000.4',
 'AC127521.1',
 'AC243829.1',
 'AC244502.1',
 'ACOT11',
 'ACRBP',
 'ACSM1',
 'ACTB',
 'ACTN1',
 'ACY3',
 'ADA2',
 'ADAM12',
 'ADAM28',
 'ADAM8',
 'ADAMTS7',
 'ADCY6',
 'ADCYAP1',
 'ADGRA2',
 'ADGRE2',
 'ADGRG6',
 'ADHFE1',
 'AFF2',
 'AFF3',
 'AGAP1',
 'AGO4',
 'AHNAK',
 'AJ009632.2',
 'AK9',
 'AKAP12',
 'AKR1C3',
 'AL034397.3',
 'AL035681.1',
 'AL078600.1',
 'AL109840.2',
 'AL110114.1',
 'AL132709

### Obs DataFrame

In [5]:
adata.obs

,n_counts,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,initial_size_spliced,initial_size_unspliced,initial_size,S_score,G2M_score,phase,fractions_u,leiden
AAACAGCCAAACCTTG-1,4234.334961,1497,3384.0,0.0,0.0,3384,1563,3384.0,-0.634671,-0.215746,G1,0.308031,Granulocyte
AAACAGCCACCCTCAC-1,4246.441406,1673,3251.0,0.0,0.0,3251,1904,3251.0,0.423275,-0.362225,S,0.301643,Erythrocyte
AAACATGCAATCGCAC-1,4228.449219,1652,3272.0,0.0,0.0,3272,1849,3272.0,-0.801623,-0.463818,G1,0.374815,Erythrocyte
AAACATGCACAGCCTG-1,4223.146973,3637,10265.0,0.0,0.0,10265,3769,10265.0,0.408826,0.540197,G2M,0.290607,Prog MK
AAACATGCAGAGGCTA-1,4230.780762,1572,3526.0,0.0,0.0,3526,1792,3526.0,0.400493,-0.121910,S,0.318393,MPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTTAAAC-1,4194.219238,1642,3244.0,0.0,0.0,3244,1723,3244.0,0.000000,-0.201697,S,0.340409,HSC
TTTGTTGGTGTTGTGA-1,4216.825195,2306,5480.0,0.0,0.0,5480,2348,5480.0,-0.291098,-0.117548,G1,0.345615,MEP
TTTGTTGGTTACCTGT-1,4240.065430,1664,3310.0,0.0,0.0,3310,1678,3310.0,0.000000,0.155516,G2M,0.304211,MEP
TTTGTTGGTTCCGGGA-1,4253.901855,2193,5188.0,0.0,0.0,5188,1870,5188.0,0.296031,0.044591,S,0.322785,Erythrocyte


In [6]:
df = adata.obs

In [7]:
df.columns

Index(['n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'initial_size_spliced', 'initial_size_unspliced',
       'initial_size', 'S_score', 'G2M_score', 'phase', 'fractions_u',
       'leiden'],
      dtype='object')

In [8]:
df['leiden'].unique().tolist()

['Granulocyte',
 'Erythrocyte',
 'Prog MK',
 'MPP',
 'GMP',
 'HSC',
 'LMPP',
 'MEP',
 'Prog DC',
 'Platelet',
 'Prog B']

In [9]:
# double check at least PROG DC, this was the primary one that was
# unclear in identifying

# all of these values vall under Hematopoietic cell
# 553 and 1012 not in cell census

mapping_dict = {'Granulocyte' : 'CL:0000094',
                'Erythrocyte' : 'CL:0000232',
                'Prog MK' : 'CL:0000553',
                'MPP' : 'CL:0000837',
                'GMP' : 'CL:0000557',
                'HSC' : 'CL:0000037',
                'LMPP' : 'CL:0000936',
                'MEP' : 'CL:0000050',
                'Prog DC' : 'CL:0001012',
                'Platelet' : 'CL:0000233',
                'Prog B' : 'CL:0000826'}    
    

df['cell_type'] = df['leiden'].map(mapping_dict)

In [10]:
df['cell_type'].value_counts()

cell_type
CL:0000936    1816
CL:0000232    1606
CL:0000837    1605
CL:0000037    1529
CL:0000557    1344
CL:0000050    1246
CL:0000553    1240
CL:0000094     661
CL:0001012     492
CL:0000826      36
CL:0000233      30
Name: count, dtype: int64

In [11]:
df.shape

(11605, 14)

936 and 233 are the only leaf nodes, so that amounts to 1846 cells, which is 16% of the total sample.

## Load the PyTorch Model

In [11]:
input_dim = 19966 # need to check if we mean 19966??????

# number of neurons for hidden layers
hidden_layer_1 = 256
hidden_layer_2 = 128

# number of classes (unique of y)
output_dim = 52

In [12]:
class Network2(nn.Module):
    def __init__(self):
        super(Network2, self).__init__()
        self.linear1 = nn.Linear(input_dim,hidden_layer_1)
        self.linear2 = nn.Linear(hidden_layer_1,hidden_layer_2)
        self.linear3 = nn.Linear(hidden_layer_2,output_dim)
        self.bn1 = nn.BatchNorm1d(hidden_layer_1)
        self.bn2 = nn.BatchNorm1d(hidden_layer_2)
        
    def forward(self,x):
        x = self.linear1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.softmax(x,dim=1)
        return x

In [13]:
clf = Network2()

# use map_location if model was trained on GPU but is being loaded on CPU
os.chdir('/home/fujoshua/cell_classification')

#model = torch.load('30_Oct_best_model',map_location=torch.device('cpu'))
clf.load_state_dict(torch.load('9_Nov_best_model',map_location=torch.device('cpu')))
# set model to eval mode
clf.eval()


Network2(
  (linear1): Linear(in_features=19966, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=52, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

## Map the genes in the training data to the genes in the hand-annotated data

The hand-annotated data set needs to be structured the same as the Cell_Census data, particularly with respect to the order of the genes in the dataframe. We can use ENSEMBL GENE IDs to transform the hand-annotated data to match the cell_census data

- Gene information in the training data from Cell_Census is stored under var feature_id
- Gene information in the hand-annotated data is stored under var Accession

In [14]:
# get the training data
#train_adata = ad.read_h5ad('small_1044_900') # 42000 cells, 60k genes, including noncoding ones


os.chdir('/scratch/welchjd_root/welchjd99/fujoshua')
train_adata = ad.read_h5ad('24Oct_hematopoietic_cells_p2') # 472k cells, 20k genes, part of the full hema data set



In [15]:
train_adata

AnnData object with n_obs × n_vars = 472343 × 19966
    obs: 'cell_type_ontology_term_id', 'assay', 'is_primary_data'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

In [55]:
train_adata.var['feature_name']

0           AP006222.1
1            LINC00115
2               FAM41C
3               SAMD11
4                NOC2L
             ...      
60659    RP4-669P10.21
60660    RP11-852E15.3
60661     RP5-973N23.5
60662    RP11-553N16.6
60663     RP13-546I2.2
Name: feature_name, Length: 60664, dtype: object

In [15]:
train_genes = train_adata.var['feature_id'].to_list()

In [16]:
train_genes

['ENSG00000228463',
 'ENSG00000225880',
 'ENSG00000230368',
 'ENSG00000187634',
 'ENSG00000188976',
 'ENSG00000187961',
 'ENSG00000187583',
 'ENSG00000187642',
 'ENSG00000188290',
 'ENSG00000187608',
 'ENSG00000188157',
 'ENSG00000131591',
 'ENSG00000162571',
 'ENSG00000186891',
 'ENSG00000186827',
 'ENSG00000078808',
 'ENSG00000176022',
 'ENSG00000184163',
 'ENSG00000160087',
 'ENSG00000162572',
 'ENSG00000131584',
 'ENSG00000169972',
 'ENSG00000127054',
 'ENSG00000224051',
 'ENSG00000107404',
 'ENSG00000162576',
 'ENSG00000175756',
 'ENSG00000221978',
 'ENSG00000242485',
 'ENSG00000235098',
 'ENSG00000179403',
 'ENSG00000215915',
 'ENSG00000160072',
 'ENSG00000197785',
 'ENSG00000160075',
 'ENSG00000215014',
 'ENSG00000228594',
 'ENSG00000197530',
 'ENSG00000189409',
 'ENSG00000248333',
 'ENSG00000189339',
 'ENSG00000008128',
 'ENSG00000215790',
 'ENSG00000008130',
 'ENSG00000078369',
 'ENSG00000178821',
 'ENSG00000187730',
 'ENSG00000067606',
 'ENSG00000162585',
 'ENSG00000157933',


In [21]:
# create a dictionary where each key is a gene in train_gene and
# the value is the index in the list

train_gene_dict = {k: v for v, k in enumerate(train_genes)}

In [22]:
train_gene_dict

{'ENSG00000228463': 0,
 'ENSG00000225880': 1,
 'ENSG00000230368': 2,
 'ENSG00000187634': 3,
 'ENSG00000188976': 4,
 'ENSG00000187961': 5,
 'ENSG00000187583': 6,
 'ENSG00000187642': 7,
 'ENSG00000188290': 8,
 'ENSG00000187608': 9,
 'ENSG00000188157': 10,
 'ENSG00000131591': 11,
 'ENSG00000162571': 12,
 'ENSG00000186891': 13,
 'ENSG00000186827': 14,
 'ENSG00000078808': 15,
 'ENSG00000176022': 16,
 'ENSG00000184163': 17,
 'ENSG00000160087': 18,
 'ENSG00000162572': 19,
 'ENSG00000131584': 20,
 'ENSG00000169972': 21,
 'ENSG00000127054': 22,
 'ENSG00000224051': 23,
 'ENSG00000107404': 24,
 'ENSG00000162576': 25,
 'ENSG00000175756': 26,
 'ENSG00000221978': 27,
 'ENSG00000242485': 28,
 'ENSG00000235098': 29,
 'ENSG00000179403': 30,
 'ENSG00000215915': 31,
 'ENSG00000160072': 32,
 'ENSG00000197785': 33,
 'ENSG00000160075': 34,
 'ENSG00000215014': 35,
 'ENSG00000228594': 36,
 'ENSG00000197530': 37,
 'ENSG00000189409': 38,
 'ENSG00000248333': 39,
 'ENSG00000189339': 40,
 'ENSG00000008128': 41,
 '

In [17]:
hand_genes = adata.var['Accession'].to_list()

In [20]:
hand_genes

['ENSG00000183044',
 'ENSG00000165029',
 'ENSG00000108846',
 'ENSG00000125257',
 'ENSG00000140526',
 'ENSG00000099204',
 'ENSG00000251259',
 'ENSG00000225792',
 'ENSG00000287643',
 'ENSG00000269696',
 'ENSG00000286997',
 'ENSG00000267549',
 'ENSG00000284959',
 'ENSG00000286656',
 'ENSG00000287263',
 'ENSG00000270049',
 'ENSG00000247121',
 'ENSG00000257176',
 'ENSG00000235192',
 'ENSG00000256377',
 'ENSG00000287127',
 'ENSG00000229308',
 'ENSG00000285354',
 'ENSG00000286707',
 'ENSG00000286481',
 'ENSG00000267174',
 'ENSG00000224739',
 'ENSG00000287125',
 'ENSG00000257764',
 'ENSG00000186019',
 'ENSG00000258449',
 'ENSG00000256948',
 'ENSG00000272368',
 'ENSG00000253582',
 'ENSG00000257239',
 'ENSG00000254343',
 'ENSG00000240497',
 'ENSG00000283839',
 'ENSG00000251555',
 'ENSG00000251584',
 'ENSG00000273247',
 'ENSG00000250696',
 'ENSG00000262823',
 'ENSG00000274767',
 'ENSG00000251002',
 'ENSG00000162390',
 'ENSG00000111644',
 'ENSG00000166743',
 'ENSG00000075624',
 'ENSG00000072110',


In [18]:
hand_in_census = [gene for gene in hand_genes if gene in train_genes]

In [19]:
len(hand_in_census)

899

### Build AnnData Object that matches the CellCensus Structure

We need a function that will transform an external AnnData object to match the structure of the CellCensus data. Most importantly, we need to match the gene order.

In [16]:
def build_data_object(external_adata,external_adata_gene_id,train_adata,train_gene_id):
    '''
    This function takes an external AnnData object and transforms it, as needed, to 
    match the structure of Cell Census AnnData objects. 
    
    external_adata : AnnData object
        data that you want to transform
        
    external_adata_gene_id : string
        string with the var name of the EMSEMBL genes
        
    train_adata : AnnData object
        this object contains the data format we want to match
    
    train_gene_id : string
        string with the var name of the ENSEMBL genes
    
    '''    
    
    # convert the new_adata object (the one to be
    # transformed) to a DataFrame
    external_adata_df = external_adata.to_df(layer=None)
    
    # change the column names of this dataframe 
    # to the ENSEMBL Gene IDs
    # get list of genes from the var part of the adata object
    # these are stored in the same order as the HGNC names, so we
    # can just replace the column names
    # can use BioMart to double check mappings
    # https://useast.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000111644;r=12:6638075-6647433
    
    
    external_listofensemblIDs = external_adata.var[external_adata_gene_id].to_list()

    external_adata_df.columns = external_listofensemblIDs

    
    # get the list of genes in the traing object
    train_genes = train_adata.var[train_gene_id].to_list()
    
    ### create a dictionary where each key is a gene in census_gene and
    ### the value is the index in the list

    ###train_gene_dict = {k: v for v, k in enumerate(train_genes)}
    ###print(list(train_gene_dict.keys())[list(train_gene_dict.values()).index('ENSG00000237940')])  # Prints george


    # we have the external df with ENSEMBL #s for columns,
    # now, we need to create a new dataset, where the column order comes from
    # the training data, IF the column name is in adata_df, use that, else input 0s
    
    # get list of all genes in train_genes NOT IN external_listofensemblIDs
    missing_genes = [x for x in train_genes if x not in external_listofensemblIDs]

    
    # there are extra genes in external_listofensemblIDs that are not in train_genes
    # get a list of those to remove
    extra_genes = [x for x in external_listofensemblIDs if x not in train_genes]  

    
    external_adata_df.drop(columns=extra_genes,inplace=True)

    # create an new Dataframe with genes from missing_genes, expression values
    # of 0 
    genes_to_add_df = pd.DataFrame(0,index=external_adata_df.index,columns=missing_genes)

    # add these genes to the existing DataFrame, with expression values of 0
    #adata_df[missing_genes] = 0
    combined_df = pd.concat([external_adata_df,genes_to_add_df],axis=1)
    
    # take combined_df and re-order the columns to match the
    # order from census_adata
    transformed_df = combined_df[train_genes]
    #print(transformed_df.shape)
    
    # don't need to do this at the moment, saving the idea for later
    # convert to AnnData object
    # from https://stackoverflow.com/questions/65117950/how-do-i-create-anndata-object-from-a-pandas-data-frame-count-matrix
    #new_adata = ad.AnnData(X = transformed_df.values,
    #                    obs = transformed_df.index,
    #                    var = transformed_df.columns)

    return(transformed_df)
    
    

'ENSG00000121410', 'ENSG00000148584', 'ENSG00000175899'

In [17]:
train_gene_id = 'feature_id'

external_adata_gene_id = 'gene_ids'


external_expanded_df = build_data_object(merged_data,external_adata_gene_id,train_adata,train_gene_id)


In [18]:
external_expanded_df.shape


(11605, 19966)

In [16]:
external_expanded_df.sum(axis=0)#.value_counts()

ENSG00000121410    2501.0
ENSG00000148584       1.0
ENSG00000175899    3155.0
ENSG00000166535      60.0
ENSG00000184389      25.0
                    ...  
ENSG00000288711       0.0
ENSG00000288712       0.0
ENSG00000288715       0.0
ENSG00000288716       0.0
ENSG00000288721       0.0
Length: 19966, dtype: float64

In [17]:
external_expanded_df

,ENSG00000121410,ENSG00000148584,ENSG00000175899,ENSG00000166535,ENSG00000184389,ENSG00000128274,ENSG00000118017,ENSG00000094914,ENSG00000081760,ENSG00000114771,...,ENSG00000288695,ENSG00000288699,ENSG00000288706,ENSG00000288708,ENSG00000288710,ENSG00000288711,ENSG00000288712,ENSG00000288715,ENSG00000288716,ENSG00000288721
AAACAGCCAAACCTTG-1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
AAACAGCCACCCTCAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
AAACATGCAATCGCAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
AAACATGCACAGCCTG-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
AAACATGCAGAGGCTA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTTAAAC-1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGGTGTTGTGA-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGGTTACCTGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGGTTCCGGGA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
merged_data.var['gene_ids']['LINC01238']

LINC01238    ENSG00000237940
LINC01238    ENSG00000261186
Name: gene_ids, dtype: object

In [33]:
traintest = train_adata.var['feature_name']
traintest

0           AP006222.1
1            LINC00115
2               FAM41C
3               SAMD11
4                NOC2L
             ...      
60659    RP4-669P10.21
60660    RP11-852E15.3
60661     RP5-973N23.5
60662    RP11-553N16.6
60663     RP13-546I2.2
Name: feature_name, Length: 60664, dtype: object

In [40]:
traintest = train_adata.var[['feature_id', 'feature_name']]
traintest
#traintest[traintest['feature_id'] == 'ENSG00000261186']

,feature_id,feature_name
0,ENSG00000228463,AP006222.1
1,ENSG00000225880,LINC00115
2,ENSG00000230368,FAM41C
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
60659,ENSG00000288719,RP4-669P10.21
60660,ENSG00000288720,RP11-852E15.3
60661,ENSG00000288721,RP5-973N23.5
60662,ENSG00000288723,RP11-553N16.6


In [41]:
#df[df.Color.str.contains('#')]
traintest[traintest.feature_name.str.contains('_')]

,feature_id,feature_name
781,ENSG00000270066,SCARNA2_ENSG00000270066
1165,ENSG00000143248,RGS5_ENSG00000143248
1594,ENSG00000237667,LINC01115_ENSG00000237667
2386,ENSG00000128655,PDE11A_ENSG00000128655
2737,ENSG00000237940,LINC01238_ENSG00000237940
...,...,...
59547,ENSG00000278625,U6_ENSG00000278625
59549,ENSG00000278793,U6_ENSG00000278793
60065,ENSG00000287297,XXyac-YX82D3__B.1
60547,ENSG00000288601,5S_rRNA_ENSG00000288601


## Preprocess Data for Modeling

We'll follow the same process of preprocessing the data before we input it into the model. First, we need to convert the pandas dataframe to a scipy sparse CSR matrix. Then, we transform the data with log(1+data). Finally, we convert the data to PyTorch Tensors.

In [19]:
def preprocess_data(external_df):
    '''
    This function takes the input Pandas Dataframe and then:
    1) converts to a scipy sparse CSR matrix
    2) transforms the data with log(1+data)
    3) converts the data to pytorch tensors
    
    Parameters
    ----------
    external_df : pandas dataframe
        
        
    Returns
    -------
    x_data : SciPy Matrix
        scipy sparse CSR matrix
    
    '''
    
    # convert the data to a scipy sparse CSR matrix
    x_data = scipy.sparse.csr_matrix(external_df.values)
    
    # transform the data
    # np.log takes the natural log
    x_data.data = np.log(1+ x_data.data)

    # convert x_data to a pytorch tensor, starting
    # by converting it to COO formating
    # following https://stackoverflow.com/questions/50665141/converting-a-scipy-coo-matrix-to-pytorch-sparse-tensor
    # 
    x_data_coo = x_data.tocoo()

    # for some reason, doing this at once loosed 70 vlaues, so we'll
    # do it in pieces instead.
    #x_data_tensor = torch.sparse.FloatTensor(torch.LongTensor(np.vstack((x_data_coo.row,x_data_coo.col))),
    #                              torch.FloatTensor(x_data_coo.data))
    
    values = x_data_coo.data
    indices = np.vstack((x_data_coo.row, x_data_coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = x_data_coo.shape

    x_data_tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))    
    
    #print(x_data_tensor.shape)
    
    # move to device
    x_data_tensor = x_data_tensor.to(device)

    return x_data_tensor

In [20]:
formatted_data = preprocess_data(external_expanded_df)


In [21]:
19966-19896

70

In [42]:
formatted_data

tensor(indices=tensor([[    0,     0,     0,  ..., 11604, 11604, 11604],
                       [    2,    26,    53,  ..., 18829, 18839, 19130]]),
       values=tensor([1.0986, 0.6931, 0.6931,  ..., 0.6931, 0.6931, 0.6931]),
       size=(11605, 19966), nnz=40481856, layout=torch.sparse_coo)

In [43]:
external_expanded_df.shape

(11605, 19966)

## Run the data through the model

In [21]:
external_outputs = clf(formatted_data.float()) # might need to change to X_train.float()


In [22]:
external_outputs

tensor([[3.4109e-01, 2.9599e-03, 3.7882e-02,  ..., 2.4208e-03, 2.1746e-03,
         2.0435e-03],
        [8.1442e-03, 4.1980e-03, 2.6317e-03,  ..., 7.1323e-04, 6.8925e-04,
         7.1886e-04],
        [5.0874e-02, 4.1077e-03, 3.0669e-03,  ..., 1.8077e-03, 1.1431e-03,
         8.9748e-04],
        ...,
        [8.5578e-01, 3.3353e-03, 8.0262e-03,  ..., 1.2232e-03, 6.3361e-04,
         8.4421e-04],
        [2.3135e-01, 1.4750e-02, 9.2650e-03,  ..., 2.0007e-03, 1.2215e-03,
         5.2859e-03],
        [4.4644e-01, 6.1004e-03, 1.4103e-02,  ..., 9.4694e-03, 1.1426e-02,
         6.2960e-03]], grad_fn=<SoftmaxBackward0>)

In [47]:
# get the predictions
__, predicted_external_outputs = external_outputs.max(dim=1)            


In [48]:
predicted_external_outputs.unique(return_counts=True)

(tensor([ 0,  1,  2,  4,  5,  6,  7,  8, 10, 11, 12, 15, 16, 19, 21, 22, 23, 28,
         30, 34, 35, 36, 37, 40, 41, 45, 47, 49]),
 tensor([5031,    8,    4,    9,  893,    8,   17, 1471,   32,   15,  200,  589,
         2217,  843,   38,    8,   46,    1,   63,   24,    1,    7,   14,    1,
            9,   22,    1,   33]))

In [49]:
external_outputs.shape

torch.Size([11605, 52])

## Compare the predicted outputs to the hand-annotated data.

We'll want to split the dataset into predicted leaf and internal nodes. Then we'll calculate the leaf accuracy and internal nodes accuracy separately. Finally, for the internal nodes, we'll want to check if we can make a 'better' prediction.

In [23]:
def split_data(external_outputs, leaf_index_values,internal_index_values):
    '''
    
    
    '''
    
    leaf_external_outputs = external_outputs[leaf_index_values,:]
    
    internal_external_outputs = external_outputs[internal_index_values,:]
    
    return(leaf_external_outputs,internal_external_outputs)
    
    
    

In [24]:
leaf_external_outputs,internal_external_outputs = split_data(external_outputs, 
                                                             leaf_index_values,internal_index_values)



In [25]:
leaf_external_outputs.shape

torch.Size([1816, 52])

In [26]:
# mapping dict should come from cell_classification, but I'll make it up here for now

mapping_dict_encoding = {'CL:0000936' : 0}

In [42]:
def check_leaf_predictions(leaf_external_outputs, leaf_target_values,mapping_dict):
    '''
    
    '''
    
    # get the predictions for the leaf external outputs
    
    # get the predictions
    __, predicted_leaf_external_outputs = leaf_external_outputs.max(dim=1)
    
    
    # encode leaf target values using the same method 
    target_value_list = list(map(mapping_dict_encoding.get,leaf_targets))
    
    # convert to a tensor
    target_values_tensor = torch.LongTensor(target_value_list)
    
    # calculate the accuracy
    correct_leaf = (predicted_leaf_external_outputs == target_values_tensor).sum().item()
    #print(torch.sum(target_value_list == target_values_tensor))

    accuracy_leaf = (correct_leaf / predicted_leaf_external_outputs.shape[0] * 100.)
    print(accuracy_leaf)
    
    return(accuracy_leaf)
    
    

In [43]:
accuracy_leaf = check_leaf_predictions(leaf_external_outputs,leaf_targets,mapping_dict_encoding)

56.77312775330397


In [48]:
def output_probability_tensor(outputs,ontology_df):
    '''
    Function to convolve the predicted classification outputs with the ontology heirarchy to
    get predicted normalized probabilities for all parents. 
    Precursur to loss calculation
    
    Parameters
    ----------
    outputs : tensor
        PyTorch tensor of shape [a,b] where a = number of cells and b = number of target leafs
        This tensor is the result of the classification in the neural network
                
    ontology_df : pandas dataframe
        pandas dataframe where rows are parent labels and columns are leafs
        values indicate if parent node is an ancestor of leaf node


    Outputs
    -------
    sum_probability_tensor : tensor
        PyTorch tensor of shape [c,a] where c = number of cell ontology IDs and a = number of cells
        Each entry is the summed predicted probability for that cell ID

    '''
        
    # convert the dataframe to a pytorch tensor
    ontology_tensor = torch.FloatTensor(ontology_df.values)
    ontology_tensor = ontology_tensor.to(device)
        
    # convolve the ontology tensor with the predicted outputs
    # ontology tensor is shape ij, where i = parent IDs, j = probability for leaf IDs
    # output tensor is shape kj, where k = number of cells classified, j = probability for leaf IDs
    # probability tensor is shape ijk
    
    # if there is only a single column for the ontology, change shape to match expected value
    if len(ontology_tensor.shape) == 1:
        ontology_tensor = ontology_tensor.unsqueeze(1)    
    
    probability_tensor = torch.einsum('ij,kj->ijk',ontology_tensor,outputs)
    #print('prob_tensor', probability_tensor.shape)
    
    # sum across leafs to get the predicted probability, by cell, for each
    # parent 
    # sum_probability_tensor is shape ik, where i = parent IDs, k = number of cells
    sum_probability_tensor = torch.sum(probability_tensor,dim=1,dtype=float)
    #print('sum_prob', sum_probability_tensor.shape)
        
    # ensure that the max value is 1 because of floating point issues
    # if we don't do this, we can run into errors with the binary cross entropy
    sum_probability_tensor = torch.where(sum_probability_tensor > 1, 1.0, sum_probability_tensor )
    
    return sum_probability_tensor

In [75]:
def check_internal_predictions(internal_external_outputs,ontology_leaf_df,internal_targets):
    '''
    Check the predictions for the internal nodes 
    
    Procedure:
    1) take leaf predicted probabilities
    2) USE output_probability_tensor function from cell_classification:
         use the ontology_leaf_df from cell_classification to calculate the predicted probabilities
         for each internal node 
    3) from the known target value, select that probability for each cell
    4) compare predicted probability to a threshold value to count as right/wrong
    '''
    
    internal_probability_tensor = output_probability_tensor(internal_external_outputs,ontology_leaf_df)
    
    print(internal_probability_tensor.shape)
    
    # for internal_probability_tensor, each column is a different cell, and each row 
    # a different cell ontology type,
    # for each column, we need to select the value that corresponds to the hand-annotated
    # value
    
    ontology_df_index_list = ontology_leaf_df.index.to_list()

    target_index = []

    for x in internal_targets:
        target_index.append(ontology_df_index_list.index(x))



In [57]:
# load ontology_df from cell_classification


os.chdir('/home/fujoshua/cell_classification')
ontology_leaf_df = pd.read_csv('ontology_leaf_df_10Nov.csv',index_col=0)
#ontology_df

In [76]:
check_internal_predictions(internal_external_outputs,ontology_leaf_df,internal_targets)

torch.Size([88, 9789])


ValueError: 'CL:0000553' is not in list

In [73]:
ontology_df_index_list = ontology_leaf_df.index.to_list()

ontology_df_index_list.index('CL:0000232')
#[ontology_leaf_df.index == 'CL:0000094']

target_index = []

for x in internal_targets:
    target_index.append(ontology_df_index_list.index(x))
    
target_index

ValueError: 'CL:0000553' is not in list

In [72]:
ontology_leaf_df.head(8)

,CL:0000583,CL:0000861,CL:0000765,CL:0000903,CL:0000910,CL:0000051,CL:0000817,CL:0001029,CL:0000050,CL:0000547,...,CL:0000810,CL:0000811,CL:0002057,CL:0002397,CL:0000800,CL:0001058,CL:0000818,CL:0001043,CL:0001049,CL:0000776
CL:0001012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CL:0000893,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
CL:0000791,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
CL:0000764,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
CL:0000081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
CL:0000738,1,1,0,1,1,0,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
CL:0000909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CL:0000232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
internal_targets

['CL:0000094',
 'CL:0000232',
 'CL:0000232',
 'CL:0000553',
 'CL:0000837',
 'CL:0000557',
 'CL:0000037',
 'CL:0000837',
 'CL:0000094',
 'CL:0000094',
 'CL:0000553',
 'CL:0000837',
 'CL:0000557',
 'CL:0000837',
 'CL:0000557',
 'CL:0000232',
 'CL:0000553',
 'CL:0000837',
 'CL:0000553',
 'CL:0000553',
 'CL:0000232',
 'CL:0000553',
 'CL:0000037',
 'CL:0000557',
 'CL:0000037',
 'CL:0000050',
 'CL:0000232',
 'CL:0000557',
 'CL:0000557',
 'CL:0000553',
 'CL:0000553',
 'CL:0000557',
 'CL:0000837',
 'CL:0000557',
 'CL:0000837',
 'CL:0000037',
 'CL:0000094',
 'CL:0000557',
 'CL:0000557',
 'CL:0000557',
 'CL:0000232',
 'CL:0000037',
 'CL:0000094',
 'CL:0000837',
 'CL:0000837',
 'CL:0000557',
 'CL:0000232',
 'CL:0001012',
 'CL:0000050',
 'CL:0000553',
 'CL:0000050',
 'CL:0000557',
 'CL:0000037',
 'CL:0000037',
 'CL:0000094',
 'CL:0000557',
 'CL:0000232',
 'CL:0000037',
 'CL:0000557',
 'CL:0000557',
 'CL:0000037',
 'CL:0000837',
 'CL:0000232',
 'CL:0000837',
 'CL:0000837',
 'CL:0000837',
 'CL:00000

In [63]:
ontology_leaf_df

,CL:0000583,CL:0000861,CL:0000765,CL:0000903,CL:0000910,CL:0000051,CL:0000817,CL:0001029,CL:0000050,CL:0000547,...,CL:0000810,CL:0000811,CL:0002057,CL:0002397,CL:0000800,CL:0001058,CL:0000818,CL:0001043,CL:0001049,CL:0000776
CL:0001012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CL:0000893,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
CL:0000791,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
CL:0000764,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
CL:0000081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CL:0000094,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
CL:0001056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
CL:0002419,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
CL:0002191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
mapping_dict_encoding

{'CL:0000936': 0}

In [84]:
list1 = [0,1,2]
list2 = [0,3,4]


AttributeError: 'bool' object has no attribute 'sum'